# 日本語の文章をBag of Wordsにするまで

Bag of Wordsは係り受けの情報は考えず、文中の単語とその出現頻度にのみ着目した文のベクトル表現。  
分類タスクには使えるが、時系列は無視してしまっているので、生成系は工夫が必要だろう。  

また、ストップワードの除去や次元の削減は行う必要がある。

## 1. Tokenise

In [19]:
import MeCab
from MeCab import Tagger

from gensim import corpora, matutils

from typing import List, Dict

mecabでテキストを形態素解析し、.mecabで保存する。

In [10]:
with open("/Users/fukunaritakeshi/PycharmProjects/deep-learning/data/neko.txt", "r") as file:
    lines: List[str] = file.readlines()

In [51]:
lines[5]

'\u3000どこで生れたかとんと見当がつかぬ。\n'

In [52]:
tagger: Tagger = MeCab.Tagger()
sentence: str = tagger.parse(lines[5])

In [53]:
sentence

'\u3000\t記号,空白,*,*,*,*,\u3000,\u3000,\u3000\nどこ\t名詞,代名詞,一般,*,*,*,どこ,ドコ,ドコ\nで\t助詞,格助詞,一般,*,*,*,で,デ,デ\n生れ\t動詞,自立,*,*,一段,連用形,生れる,ウマレ,ウマレ\nた\t助動詞,*,*,*,特殊・タ,基本形,た,タ,タ\nか\t助詞,副助詞／並立助詞／終助詞,*,*,*,*,か,カ,カ\nとんと\t副詞,一般,*,*,*,*,とんと,トント,トント\n見当\t名詞,サ変接続,*,*,*,*,見当,ケントウ,ケントー\nが\t助詞,格助詞,一般,*,*,*,が,ガ,ガ\nつか\t動詞,自立,*,*,五段・カ行イ音便,未然形,つく,ツカ,ツカ\nぬ\t助動詞,*,*,*,特殊・ヌ,基本形,ぬ,ヌ,ヌ\n。\t記号,句点,*,*,*,*,。,。,。\nEOS\n'

In [55]:
word_list = sentence.split("\n")

In [56]:
word_list

['\u3000\t記号,空白,*,*,*,*,\u3000,\u3000,\u3000',
 'どこ\t名詞,代名詞,一般,*,*,*,どこ,ドコ,ドコ',
 'で\t助詞,格助詞,一般,*,*,*,で,デ,デ',
 '生れ\t動詞,自立,*,*,一段,連用形,生れる,ウマレ,ウマレ',
 'た\t助動詞,*,*,*,特殊・タ,基本形,た,タ,タ',
 'か\t助詞,副助詞／並立助詞／終助詞,*,*,*,*,か,カ,カ',
 'とんと\t副詞,一般,*,*,*,*,とんと,トント,トント',
 '見当\t名詞,サ変接続,*,*,*,*,見当,ケントウ,ケントー',
 'が\t助詞,格助詞,一般,*,*,*,が,ガ,ガ',
 'つか\t動詞,自立,*,*,五段・カ行イ音便,未然形,つく,ツカ,ツカ',
 'ぬ\t助動詞,*,*,*,特殊・ヌ,基本形,ぬ,ヌ,ヌ',
 '。\t記号,句点,*,*,*,*,。,。,。',
 'EOS',
 '']

In [57]:
surface_list: List[str] = []
for word in word_list:
    surface: str = word.split("\t")[0]
    surface_list.append(surface)

In [58]:
surface_list

['\u3000',
 'どこ',
 'で',
 '生れ',
 'た',
 'か',
 'とんと',
 '見当',
 'が',
 'つか',
 'ぬ',
 '。',
 'EOS',
 '']

In [59]:
surface_list.remove("EOS")
surface_list.remove("")
surface_list.remove("\u3000")

In [61]:
surface_list

['どこ', 'で', '生れ', 'た', 'か', 'とんと', '見当', 'が', 'つか', 'ぬ', '。']

これまでの一連の流れをtokenize関数として定義する。

In [12]:
def tokenize(*, text_file_path: str) -> List[List[str]]:
    
    with open(text_file_path, "r") as file:
        lines: List[str] = file.readlines()
    
    tagger: Tagger = MeCab.Tagger()    
    surface_lists: List[List[str]] = []
    for line in lines:
        sentence: str = tagger.parse(line)
        word_list = sentence.split("\n")
        
        surface_list: List[str] = []
        for word in word_list:
            surface: str = word.split("\t")[0]
            if not(surface == "EOS" or surface == "" or surface == "\u3000"):
                surface_list.append(surface)
        
        # 空行や一単語のものは除外する
        if len(surface_list) > 1:
            surface_lists.append(surface_list)   
        
    return surface_lists

In [13]:
surface_lists: List[List[str]] = tokenize(text_file_path="/Users/fukunaritakeshi/PycharmProjects/deep-learning/data/neko.txt")

In [14]:
surface_lists[6]

['この',
 '書生',
 'という',
 'の',
 'は',
 '時々',
 '我々',
 'を',
 '捕え',
 'て',
 '煮',
 'て',
 '食う',
 'という',
 '話',
 'で',
 'ある',
 '。']

## 2. Bag of Words

ライブラリGensimを用いて特徴語の辞書をつくる。

In [81]:
dictionary = corpora.Dictionary(surface_lists)

辞書の保存や読み込みもできる。

In [90]:
dictionary_path: str = "/Users/fukunaritakeshi/PycharmProjects/deep-learning/data/neko_dictionary.txt"
dictionary.save_as_text(dictionary_path)

In [98]:
dictionary = corpora.Dictionary.load_from_text(dictionary_path)

特徴ベクトルを作成。

辞書中の単語idと頻度のタプルとなっている。

In [118]:
print(surface_lists[503])
feature_vector_tuple = dictionary.doc2bow(surface_lists[503])
print(feature_vector_tuple)

['人間', 'の', '日記', 'の', '本', '色', 'は', 'こう', '云う', '辺', 'に', '存する', 'の', 'かも', '知れ', 'ない', '。']
[(0, 1), (3, 1), (35, 1), (53, 3), (72, 1), (111, 1), (238, 1), (263, 1), (268, 1), (354, 1), (424, 1), (436, 1), (1377, 1), (2207, 1), (2291, 1)]


ベクトルの表現に変換する。

In [116]:
feature_vector = list(matutils.corpus2dense([feature_vector_tuple], num_terms=len(dictionary)).T[0])
feature_vector[:20]

[1.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0]

ここまでの流れをfeature_vector関数にし、全テキストに適用する。

In [15]:
def vectorize(*, token_lists: List[List[str]]) -> List[List[float]]:
    
    dictionary = corpora.Dictionary(token_lists)
    
    feature_vector_list: List[List[float]] = []
    for token_list in token_lists:
        feature_vector_tuple = dictionary.doc2bow(token_list)
        feature_vector: List[float] = list(matutils.corpus2dense([feature_vector_tuple], num_terms=len(dictionary)).T[0])
        feature_vector_list.append(feature_vector)
        
    return feature_vector_list

In [16]:
feature_vector_list: List[List[float]] = vectorize(token_lists=surface_lists)

In [17]:
feature_vector_list[500][:10]

[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]